In [2]:
from neuron import h
import os
import matplotlib.pyplot as plt

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))




#First compile our modfiles.
if os.system('nrnivmodl modfiles/') == 0:
    print("Compilation Succeeded.")
else:
    print("Compilation Failed.")


Compilation Succeeded.


In [3]:
h.load_file('stdrun.hoc')

#Define simumulation parameters.
SIMRUNTIME = 1500
DELAY      = 400
INJTIME    = 500
I_INJ      = 0.2

h.tstop    = SIMRUNTIME
h.v_init   = -75

#Now set up vectors to record the membrane potential and the time.
potential = h.Vector()
time      = h.Vector()

In [6]:
import numpy as np

tonic_soma = h.Section(name = 'tonic_soma')

tonic_iclamp = h.IClamp(tonic_soma(0.5))
tonic_iclamp.dur   = INJTIME
tonic_iclamp.amp   = I_INJ
tonic_iclamp.delay = DELAY

#We will make a super simple one compartement tonic spiking cell.
tonic_soma.nseg = 1
tonic_soma.L    = 25
tonic_soma.diam = 24.75

#Now insert the two segregated modules into the cell.
#First insert the passive module.
tonic_soma.insert('leakCA3')
tonic_soma.insert('hdCA3')

#Now insert the spiking module.
tonic_soma.insert('natCA3')
tonic_soma.insert('kdrCA3')

#record time and membrane potential for tonic cell.
#Record the membrane variables.
time.record(h._ref_t) 
potential.record(tonic_soma(0.5)._ref_v)

#Plot membrane potential of tonic cell.
def plot_membrane():
    plt.close()
    plt.figure(figsize = (20,5))
    plt.plot(time, potential)
    plt.xlabel('Time')
    plt.ylabel('Membrane Potential')
    plt.show()

def run_tonic_model(amp, gna, gkdr, gh, mseg, nseg, lseg):
    tonic_iclamp.amp = amp

    tonic_soma.glbar_leakCA3 = 1.53*3e-5
    tonic_soma.gbar_natCA3   = gna
    tonic_soma.mseg_natCA3   = mseg
    tonic_soma.gbar_kdrCA3   = gkdr
    tonic_soma.nseg_kdrCA3   = nseg
    tonic_soma.ghdbar_hdCA3  = gh
    tonic_soma.lseg_hdCA3    = lseg
    
    h.run()

    plot_membrane()

    plot_gating(mseg, nseg, lseg)

#Plot the gating curves.
def plot_gating(mseg, nseg, lseg):
    
    def m_curve(v):
        f = 0
        if(v < mseg):
            f = 0
        else: 
            f = 1.0/ (1.0 + np.exp( (-v - 35.5) / 7.2))
        return f

    def l_curve(v):
        f = 0
        if(v > lseg):
            f = 0
        else:
            f = 1.0/(1.0 + np.exp( (v + 62.4) / 8.0))
        return f
    
    def n_curve(v):
        f = 0
        if(v < nseg):
            f = 0
        else: 
            f = 1.0/(1.0 + np.exp( (0 - v) / 11.0))
        return f

    voltage_domain = np.arange(-80.0, 40.0, 0.1)

    #Now plot all three of the curves.
    plt.close()
    plt.figure(figsize = (20,5))
    plt.plot(voltage_domain, [m_curve(i) for i in voltage_domain], label= 'm_curve')
    plt.plot(voltage_domain, [l_curve(i) for i in voltage_domain], label= 'l_curve')
    plt.plot(voltage_domain, [n_curve(i) for i in voltage_domain], label= 'n_curve')
    plt.xlabel('Voltage')
    plt.ylabel('Probability')
    plt.legend(loc="upper right")
    plt.show()



#Set the default parameters for the simulations.

default_tonic = [I_INJ, 0.065, 0.065, 1.0e-5, -52.5, -55, -62.4]

'''
run_tonic_model(default_tonic[0], 
                default_tonic[1],
                default_tonic[2],
                default_tonic[3],
                default_tonic[4],
                default_tonic[5],
                default_tonic[6] )
'''

'\nrun_tonic_model(default_tonic[0], \n                default_tonic[1],\n                default_tonic[2],\n                default_tonic[3],\n                default_tonic[4],\n                default_tonic[5],\n                default_tonic[6] )\n'

In [7]:
#Create the all in one interactive plot.
import ipywidgets as widgets
from ipywidgets import HBox,VBox,Label,Layout
from IPython.display import display
%matplotlib inline

#Make a list of all widgets.
#Contains: amp slider, g_nat slider, g_kdr slider, g_h slider, m_seg, n_seg, l_seg, .
sliders_tonic = []
sliders_tonic.append(widgets.FloatSlider(value = default_tonic[0], min = 0.1, max = 1.0, step = 0.1, continuous_update=False, readout_format='.1f'))
sliders_tonic.append(widgets.FloatSlider(value = default_tonic[1], min = 0.001, max = 0.36, step = 0.00359, continuous_update=False, readout_format='.1e'))
sliders_tonic.append(widgets.FloatSlider(value = default_tonic[2], min = 6.0e-4, max = 0.18, step = 0.001794, continuous_update=False, readout_format='.1e'))
sliders_tonic.append(widgets.FloatSlider(value = default_tonic[3], min = 1.0e-5, max = 2.3e-4, step = 2.2e-6, continuous_update=False, readout_format='.1e'))
sliders_tonic.append(widgets.FloatSlider(value = default_tonic[4], min =-60, max = -50, step = 0.1, continuous_update=False, readout_format='.1f')) #need to update segregation ranges
sliders_tonic.append(widgets.FloatSlider(value = default_tonic[5], min = -60, max = -50, step = 0.1, continuous_update=False, readout_format='.1f'))
sliders_tonic.append(widgets.FloatSlider(value = default_tonic[6], min = -70, max = -60, step = 0.1, continuous_update=False, readout_format='.1f'))


labels_tonic = []
labels_tonic.append(Label(r'\( I_{inject}\ (nA) \)'))
labels_tonic.append(Label(r'\( g_{NaT}\ (mS/cm^2) \)'))
labels_tonic.append(Label(r'\( g_{Kdr}\ (mS/cm^2) \)'))
labels_tonic.append(Label(r'\( g_{h}\ (mS/cm^2) \)'))
labels_tonic.append(Label(r'\( m_{seg}\ (mV) \)'))
labels_tonic.append(Label(r'\( n_{seg}\ (mv) \)'))
labels_tonic.append(Label(r'\( l_{seg}\ (mv) \)'))

def reset_model_tonic(*args):
    for i in range(7):
        sliders_tonic[i].value = default_tonic[i]


reset = widgets.Button(description='Reset',icon='history',button_style='primary')
reset.on_click(reset_model_tonic)

ui = HBox([VBox(labels_tonic[:3]),
           VBox(sliders_tonic[:3]),
           VBox(labels_tonic[#Now set up vectors to record the membrane potential and the time.
potential = h.Vector()
time      = h.Vector()3:7]),
           VBox(sliders_tonic[3:7]),
           reset
          ])



#def run_tonic_model(amp, gna, gkdr, gh, mseg, nseg, lseg):

out = widgets.interactive_output(run_tonic_model, {'amp':sliders_tonic[0],'gna':sliders_tonic[1],'gkdr':sliders_tonic[2],'gh':sliders_tonic[3],'mseg':sliders_tonic[4],'nseg':sliders_tonic[5],'lseg':sliders_tonic[6]})

display(ui,out)

Output()